<a href="https://colab.research.google.com/github/BenxiaHu/DeepSilencer/blob/master/CNN_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math
import os
import torch
from torch import nn, Tensor
import torch.optim as optim
import numpy as np
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Define a toy dataset (replace with your own dataset)
# For this example, we'll use random sequences and labels
num_sequences = 2000
sequence_length = 200
num_classes = 2

sequences = np.random.choice(['A', 'C', 'G', 'T'], size=(num_sequences, sequence_length))
labels = np.random.randint(0, num_classes, size=num_sequences)

# Convert DNA sequences to one-hot encoding
def one_hot_encoding(sequences):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot_sequences = np.zeros((len(sequences), 4, len(sequences[0])))
    for i, seq in enumerate(sequences):
        for j, base in enumerate(seq):
            one_hot_sequences[i, mapping[base], j] = 1
    return one_hot_sequences

one_hot_sequences = one_hot_encoding(sequences)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(one_hot_sequences, labels, test_size=0.2, random_state=42)

ntokens = 200  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 6  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 8  # number of heads in ``nn.MultiheadAttention``
dropout = 0.4  # dropout probability

class Residual(nn.Module):
    '''Residual Block'''
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, inputs):
        return self.module(inputs) + inputs

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()
        #self.dropout = nn.Dropout(p=dropout)

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        position = torch.arange(0,max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


# CNN Block with Residual Connection
class CNNBlock(nn.Module):
    def __init__(self, input_size):
        super(CNNBlock, self).__init__()
        self.conv1 = nn.Conv1d(input_size,64, kernel_size=5,padding=2)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5,padding=2)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(2)

    def forward(self, x):
        out1 = self.conv1(x)
        out1 = self.relu1(out1)
        out1 = self.maxpool1(out1)

        out2 = self.relu2(out1)
        #print(out1.shape)
        #print(out2.shape)
        out2 = out2 + out1
        out2 = self.maxpool2(out2)
        return out2

# Define a CNN-Transformer model
class CNNTransformer(nn.Module):
    def __init__(self,input_size: int,ntoken: int,d_model: int, num_encoder_layers, num_classes, nhead=8,dropout: float = 0.5):#
        super().__init__()
        self.model_type = 'CNNTransformer'
        self.cnn_block = CNNBlock(input_size)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        # TransformerEncoderLayer and nn.Embedding
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.embedding = nn.Embedding(ntoken, d_model)  # 4 represents A, C, G, T
        self.d_model = d_model
        self.fc = nn.Sequential(
            nn.Linear(d_model, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        print(src.shape)                 # torch.Size([32, 4, 200])
        cnn_output = self.cnn_block(src)
        print(cnn_output.shape)          # torch.Size([32, 64, 50])
        # Apply nn.Embedding
        batch_size = cnn_output.size(0)
        cnn_output = cnn_output.view(batch_size, -1)  # Reshape to (batch_size, sequence_length)
        #print(math.sqrt(self.d_model))            # 8.0
        cnn_output = self.embedding(cnn_output.long()) # * math.sqrt(self.d_model)
        print(cnn_output.shape)          # torch.Size([64, 32, 50, 64])
        cnn_output = self.pos_encoder(cnn_output)

        # Transformer encoding
        transformer_output = self.transformer_encoder(cnn_output)
        avg_pooled = torch.mean(transformer_output, dim=0)
        output = self.fc(avg_pooled)
        return output


# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for training and testing
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNTransformer(input_size=4,ntoken=4,d_model=64, num_encoder_layers=6, num_classes=num_classes, nhead=8,dropout = 0.5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')


torch.Size([32, 4, 200])
torch.Size([32, 64, 50])
torch.Size([32, 3200, 64])


OutOfMemoryError: ignored